In [1]:
import boto3
from botocore.exceptions import NoCredentialsError

# Define the repository for which you want to generate the pre-signed URL containing the raw data
repository_url = "https://github.com/adiezsanchez/microglia_dna_damage"
repository_name = repository_url.split("/")[-1]

# Configure your AWS details
bucket_name = 'mhbucket'
folder_path = 'CMIC/raw_data/'
expiration = 7200  # URL expiration time in seconds

# Explicitly create a session with your profile
session = boto3.Session(profile_name='mhread')

# Initialize the S3 client using the session
s3_client = session.client('s3')

try:
    # Create and empty list to store all pre-signed URLs
    presigned_urls = []
    
    # List objects within the specified folder
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder_path)
    for obj in response.get('Contents', []):
        
        # Extract filepath from metadata
        file_path = obj['Key']
        # Extract filename
        file_name = file_path.split('/')[-1]
        # Extract MD5 hash from metadata
        s3_etag = obj['ETag']
        
        if file_path.endswith('.zip') and repository_name in file_path:
            # Generate a pre-signed URL for each .zip file
            presigned_url = s3_client.generate_presigned_url('get_object',
                                                             Params={'Bucket': bucket_name,
                                                                     'Key': file_path},
                                                             ExpiresIn=expiration)
            # Append presigned_url to empty list
            presigned_urls.append(presigned_url)
            # Print pre-seigned URLs and MD5 hash on a per file basis
            print(f"Pre-signed URL for {file_name}: {presigned_url}")
            print(f"S3 Etag for {file_name}: {s3_etag}")
            
except NoCredentialsError:
    print("Credentials not available")




Pre-signed URL for microglia_dna_damage.zip: https://brage.it.ntnu.no/mhbucket/CMIC/raw_data/microglia_dna_damage.zip?AWSAccessKeyId=AKIA8E210FD3550BED0D&Signature=1VaJe2XYmpMboShOajus2TXun04%3D&Expires=1732104743
S3 Etag for microglia_dna_damage.zip: "72b304de4d6a760f662c8a8ede15cca7-107"


In [2]:
print(presigned_urls)

['https://brage.it.ntnu.no/mhbucket/CMIC/raw_data/microglia_dna_damage.zip?AWSAccessKeyId=AKIA8E210FD3550BED0D&Signature=1VaJe2XYmpMboShOajus2TXun04%3D&Expires=1732104743']
